In [1]:
import matplotlib.pyplot as plt
import matplotlib
import scipy
import numpy as np
import pandas as pd
import statsmodels
import seaborn
import math



In [6]:
def Draw_random_resources(Num_Res, Cap_pdf_params, DC_pdf_params, Min_Cap = 0, Min_DC = 0, Max_Cap = 10, Max_DC = 1):
    # Cap is for capacity, dc is for duty cycle
    
    #pdf_params are the values of the relevant characteristic statistics that definie the pdf.  In this case it is the loc and scale params, or the mean and standard deviation
    
    # We draw three random variables that define an individual load:  It's capacity, it's duty cycle, and where it is in the duty cycle.  
    #For Future, I could add the length of the cycle in minutes as well.  

    
    #Drawing our capacity
    cap = scipy.stats.norm.rvs(loc=Cap_pdf_params['loc'], scale=Cap_pdf_params['scale'], size=Num_Res)
    
    #Drawing th loads Duty Cycle
    dc = scipy.stats.norm.rvs(loc=DC_pdf_params['loc'], scale=DC_pdf_params['scale'], size=Num_Res)
    
    for res in range(0,Num_Res):
        if cap[res] > Max_Cap:
            cap[res] = Max_Cap
        elif cap[res] < Min_Cap:
            cap[res] = Min_Cap
            
        if dc[res] > Max_DC:
            dc[res] = Max_Dc
        elif dc[res] < Min_DC:
            dc[res] = Min_DC
    
    Cap = pd.Series(cap, name = "Capacity")
    DC = pd.Series(dc, name = "Duty_Cycle")
    
    Resources = pd.concat([Cap, DC], axis = 1)
    
    return Resources

In [8]:
def Draw_load(Num_Res, Resources, min_changeDC = 0.1):
    #Determine where you are in the duty cycle at t=0.
    # This is basically drawing a random value between 0 and 1 to say that the load, at the instant of analysis, is 
    # the random decimal way through the period of the duty cycle.  
    lc0 = np.random.uniform(low=0.0, high=1.0, size=Num_Res)
    cap = Resources['Capacity'].tolist()
    dc = Resources['Duty_Cycle'].tolist()
    
    load = []
    flexUp = []
    flexDn = []
    for res in range(0,Num_Res):
        
        if lc0[res] <= dc[res]: #If the value drawn for the inital load condition is less than the duty cycle
            load.append(cap[res]) #The load is on
            if lc0[res] >= min_changeDC: #If the load is on and the initial load condition is greater than min 
                flexUp.append(cap[res]) #The resource can shed load up to it's capacity
                flexDn.append(0)
            else:
                flexUp.append(0)
                flexDn.append(0)
        else:
            load.append(0)
            if lc0[res] - dc[res] >= min_changeDC: #If the load is off and the initial load condition is greater than min plus the duty cycle
                flexUp.append(0)
                flexDn.append(cap[res]) #The resource could add load up to it's capacity
            else:
                flexUp.append(0)
                flexDn.append(0)
        
    LC0 = pd.Series(lc0, name = "Initial_Condition")
    Load = pd.Series(load, name = "TCL_Load")
    FlexUp = pd.Series(flexUp, name = "Up_Flexibility")
    FlexDn = pd.Series(flexDn, name = "Down_Flexibility")
    
    #####  THis should probably change
    Result = pd.concat([Resources, LC0, Load, FlexUp, FlexDn], axis = 1)
    #####
    
    return Result
    
    

In [3]:
#Let's Draw our loads and see what it looks like:
Num_Res = 100
Cap_params = {'loc' : 4.0, 'scale' : 0.75}
DC_params = {'loc' : 0.55, 'scale' : 0.1}
MinCap = 1.0
MaxCap = 7.0
MinDC = 0.0
MaxDC = 1.0
minChange = 0.1
#Loads = Draw_random_load(Num_Res, Cap_params, DC_params, min_changeDC = minChange, Min_Cap = MinCap, Min_DC = MinDC, Max_Cap = MaxCap, Max_DC = MaxDC)

print(sum(Loads['TCL_Load']), sum(Loads['Up_Flexibility']), sum(Loads['Down_Flexibility']))
print(Loads)

(223.43210918173151, 203.11767099874902, 141.21567200393537)
    TCL_Load  Up_Flexibility  Down_Flexibility  Capacity  Duty_Cycle  \
0   3.753751        3.753751          0.000000  3.753751    0.480711   
1   3.047245        3.047245          0.000000  3.047245    0.371079   
2   3.624688        3.624688          0.000000  3.624688    0.576637   
3   4.195593        4.195593          0.000000  4.195593    0.501636   
4   4.862249        4.862249          0.000000  4.862249    0.389086   
5   3.499310        3.499310          0.000000  3.499310    0.492156   
6   4.490641        4.490641          0.000000  4.490641    0.677982   
7   4.893388        4.893388          0.000000  4.893388    0.540105   
8   3.799299        3.799299          0.000000  3.799299    0.488120   
9   3.876215        3.876215          0.000000  3.876215    0.514341   
10  0.000000        0.000000          4.856218  4.856218    0.526095   
11  2.568900        2.568900          0.000000  2.568900    0.551154   
12 

In [5]:
def Measured_Power(True_Power, Prop_bias, Prop_noise_var, Zero_drift=0, Zero_noise_var=0, Quantization=0.00001):
    # This will apply measurement error to a power value in order to determine the measured value.  

    # We consider five types of error:
    # (1) Quantization Error -  error caused by the digitization of output 
    # (2) Zero-point Drift - While we may assume that the calibration of a meter begins 
        # with no bias at zero loading, over time there may be a small drift in that bias.
    # (3) Constant Noise - This is white noise existing in the measurement at any loading condition
    # (4) Bias -  Calibration errors or drift, this is proportional to the meter read
    # (5) Proportional Noise - treated as gaussian white noise that is proportional to the meter reading
    
    Zero_Error = Zero_drift + scipy.stats.norm.rvs(loc=0,scale=Zero_noise_var)
    Prop_Error = True_Power*(1 + Prop_bias + scipy.stats.norm.rvs(loc=0,scale=Prop_noise_var))
    Quantized_meter = Quantization*math.floor((Zero_Error+Prop_error)/Quantization)
    
    return Quantized_meter


### Some Comments: 
- The impact on the error of a **_single_** measurement between the types of error we describe (Bias vs. Noise) is indecipherable.  In order to determine the impact of each we must assign measurement error parameters and take repeated measurement snapshots with the same resource to look at the distribution of errors.  
- If we draw bias from a random distribution, then in aggregate, we don't expect it to look any different than noise unless we give it a non-zero distribution, and I suppose it could be drawn from a non-normal distribution.  
- Define meter classes such that the total error is inside the accuracy of the class
- The error  

In [4]:
scipy.stats.norm.rvs(loc=0,scale=0)

0.0

## Scenario analysis on the error parameters
- Generate random resoure: n = 10, 50, 100, 1000, 10000
- The zero-point drift and Constant Noise need to be relative to some quantity.  We will assume that all meters are 200A class, and that service voltage is 240V, single phase.  So the total quantity of constant error will be relative to 48kW.  This seems reasonable, as the amperage class of the meter will be based on the main breaker of a residential service panel, and homes with HVAC units are more likely to have a 200A panel than a 100A panel.  